Importamos modulos necesarios

In [1]:
from collections import defaultdict  # Importa defaultdict para agrupar artículos de manera eficiente
import re #importamos expreciones regulares
import unicodedata # Para normalizar texto Unicode y eliminar tildes/acentos

Se normalizan los títulos de los artículos

In [2]:
def normalize(text):
    if not text:
        return ""
    # Minúsculas
    text = text.lower()
    
    # Normalizar y eliminar acentos
    text = unicodedata.normalize("NFKD", text)
    text = "".join([c for c in text if not unicodedata.combining(c)])
    
    # Eliminar puntuación y símbolos especiales, excepto letras, números y espacios
    text = re.sub(r"[^\w\s]", "", text)
    
    # Quitar espacios innecesarios
    return text.strip()

Procesa un archivo RIS y separa los artículos en dos archivos:
        - archivos_unicos: Contiene los artículos únicos.
        - archivos_duplicados: Contiene los artículos duplicados (excepto uno de cada conjunto).

In [3]:
def process_ris_file(input_file, unique_output, duplicate_output):
    
    articles = defaultdict(list)  # Diccionario para agrupar artículos por título normalizado
    
    with open(input_file, "r", encoding="utf-8") as file:  # Abre el archivo RIS en modo lectura
        current_article, current_title = [], None  # Inicializa las variables para almacenar un artículo
        for line in file:  # Itera sobre cada línea del archivo
            if line.startswith("TY  -"):  # Detecta el inicio de un nuevo artículo
                if current_article and current_title:  # Guarda el artículo anterior si ya existe
                    articles[normalize(current_title)].append(current_article)  # Agrupa por título
                current_article, current_title = [line], None  # Reinicia el artículo actual
            else:
                current_article.append(line)  # Agrega la línea actual al artículo
                if line.startswith(("TI  -", "T1  -")):  # Detecta el título del artículo
                    current_title = line[6:].strip()  # Extrae el título removiendo el prefijo
        
        # Guarda el último artículo leído antes de finalizar
        if current_article and current_title:
            articles[normalize(current_title)].append(current_article)
    
    # Escribe los artículos únicos y duplicados en sus respectivos archivos
    with open(unique_output, "w", encoding="utf-8") as u, open(duplicate_output, "w", encoding="utf-8") as d:
        for entries in articles.values():  # Itera sobre cada grupo de artículos
            u.writelines(entries[0] + ["\n"])  # Escribe el primer artículo como único
            d.writelines(sum(entries[1:], []) + ["\n"])  # Escribe los duplicados si existen


ejecuatamos metodo que nos separa los articulos unicos y duplicados

In [4]:
# Llama a la función con los archivos de entrada y salida
process_ris_file("articulos_fusionados.ris", "articulos_unicos.ris", "articulos_duplicados.ris")  